In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

dataset = helpers.load_adult_income_dataset()

In [2]:
#INPUT: data, model, cont_feat, output
feature_names=['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'gender', 'hours_per_week']
cont_feat=['age','hours_per_week']
outcome_name='income'

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Checkbox, VBox,HBox,Label, interact,Box, FloatSlider, interact_manual,Layout
from ipywidgets import GridspecLayout

def counterfactual_explorer(dataname, modelname,feature_names,cont_feat,outcome_name):
    counterfactual_explorer.cont_feat=cont_feat
    displayselectchildren=[Label('1. Select Features to Use')]
    feature_children=[]
    for i in feature_names:
        new_check1=Checkbox(description=i, value=True, indent=False)
        feature_children.append(new_check1)
        displayselectchildren.append(new_check1)

    feature_vb = VBox(children = feature_children)
    displayselect=VBox(children=displayselectchildren)

    def f(**kwargs):
        use = list(kwargs.values())

        using=[]
        for i in feature_names:
            using.append(i)

        for i in range(len(use)):
            if use[i] is False:
                using.remove(feature_names[i])

        f.useusing=using

        #LABELS
        labels=[]
        for i in using:
            labels.append(Label(i))
        justlabels=VBox(children=labels)

        #FEATURE WEIGHTS

        b=widgets.Dropdown(options=['','Use Default Weights','Choose Your Own Weights'],value='',
                           description='2. Feature Weights:',style = {'description_width': 'initial'})
        f.weightdropdown=b
        def g(i):
            if i=='Choose Your Own Weights':
                feat_child=[]
                floattexts=[]
                for i in using:
                    new_floattext=widgets.FloatText(value=1,description=i)
                    floattexts.append(new_floattext)

                #justslider=VBox(children=sliders)
                #slidersandlabels=HBox(children=[justlabels,justslider])
                #feat_child.append(slidersandlabels)

                #weights=VBox(children=feat_child)
                weights=VBox(children=floattexts)
                display(weights)


                def weightvalues(**kwargs):
                    list1=list(kwargs.values())
                    print(list1)
                    f.weightvaluestouse=list1
                args1 = {
                    f'arg{i}': child
                    for i, child in enumerate(floattexts) }

                outtest=widgets.interactive_output(weightvalues, args1)
                #display(outtest)

            elif i=='Use Default Weights':
                print('Using Default Weights!')
            elif i=='':
                print('')

        out1 = widgets.interactive_output(g, {'i': b})
        feat=VBox(children=[b,out1])


        #QUERY
        a=widgets.Dropdown(options=['', 'From Dataset', 'Manually Enter'],
                           value='',description='3. Query Input:',style = {'description_width': 'initial'})

        def h(b):
            if b=='From Dataset':
                automatic_entry=widgets.IntText(value=1,description='Enter row number of query:',
                                                style = {'description_width': 'initial'})
                listautomatic_entry=[automatic_entry]

                display(automatic_entry)

                def queryvalues1(**kwargs):
                    list3=list(kwargs.values())
                    rowdata=[]
                    for i in dataset.loc[list3[0] , : ]:
                        rowdata.append(i)
                    f.queryvaluestouse=rowdata

                args3 = {
                    f'arg{i}': child
                    for i, child in enumerate(listautomatic_entry) }

                outtest3=widgets.interactive_output(queryvalues1, args3)

            elif b=='Manually Enter':
                inputq=[]

                inputs=[]
                index=-1
                for i in using:
                    index+=1
                    if i in cont_feat:
                        cont_text=widgets.FloatText(value=0)
                        inputs.append(cont_text)

                    else:
                        noncont_dropdown=widgets.Dropdown(options=dataset[i].unique())
                        inputs.append(noncont_dropdown)

                justinputs=VBox(children=inputs)
                inputsandlabels=HBox(children=[justlabels,justinputs])

                inputq.append(inputsandlabels)
                query=VBox(children=inputq)
                display(query)

                def queryvalues(**kwargs):
                    list2=list(kwargs.values())
                    print(list2)
                    f.queryvaluestouse=list2

                args2 = {
                    f'arg{i}': child
                    for i, child in enumerate(inputs) }

                outtest2=widgets.interactive_output(queryvalues, args2)

            elif b=='':
                print('')


        out2 = widgets.interactive_output(h, {'b': a})
        query1=VBox(children=[a,out2])

        #DISPLAYING FIRST 3 WIDGETS

        #display(HBox([displayselect,feat,query1]))

    
        grid = GridspecLayout(1, 3)#, height='100px')
        grid[0, 0] = displayselect
        grid[0, 1] = feat
        grid[0, 2] = query1
        display(grid)


    args = {
        f'arg{i}': child
        for i, child in enumerate(feature_children) 
    }
    out=widgets.interactive_output(f, args)
    display(out)




    # PROX/DIV WEIGHTS
    tune_proxdiv1 = Checkbox(description='4. Tune proximity/diversity?', indent=False)
    prox = widgets.FloatSlider(description='Proximity Weight', max=10, value=0.5,style = {'description_width': 'initial'})#default for prox in DiCE is 0.5
    div = widgets.FloatSlider(description='Diversity Weight', max=10, value=1,style = {'description_width': 'initial'})#default for div in DiCE is 1

    vb_proxdiv = VBox([prox, div])
    
    def show(d):
        if d==True:
            display(vb_proxdiv)
        elif d==False:
            print('')
        
    outtester = widgets.interactive_output(show, {'d': tune_proxdiv1})
    #tune_proxdiv=VBox([tune_proxdiv1,outtester])

    #NUMBER OF EXPLANATIONS
    num_exp = widgets.IntSlider(description='5. Num of Explanations', min=1, max=5,
                                style = {'description_width': 'initial'}, indent=False)



    #BUTTON
    button = widgets.Button(description="Generate Explanations!",layout=Layout(height='auto', width='auto'))
    button_output = widgets.Output()


    def on_button_clicked(b):
        with button_output:
            print("Generating explanations may take a few minutes...")
            print()

            #SETTING UP
            d = dice_ml.Data(dataframe=dataname, continuous_features=cont_feat, outcome_name=outcome_name)

            backend = 'TF'+tf.__version__[0] # TF2
            m = dice_ml.Model(model=modelname, backend=backend)

            exp = dice_ml.Dice(d, m)

            #Generating CFs
            query_instance = dict(zip(f.useusing, f.queryvaluestouse))

            if f.weightdropdown.value=='Use Default Weights':
                dice_exp = exp.generate_counterfactuals(query_instance,total_CFs=num_exp.value, desired_class="opposite",
                                                        features_to_vary=f.useusing, 
                                                        proximity_weight=prox.value, diversity_weight=div.value)
            elif f.weightdropdown.value=='Choose Your Own Weights':
                #putting weights into dict
                weightstouse=dict(zip(f.useusing, f.weightvaluestouse))
                dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=num_exp.value, desired_class="opposite",
                                                        features_to_vary=f.useusing, feature_weights=weightstouse,
                                                        proximity_weight=prox.value, diversity_weight=div.value)

            counterfactual_explorer.dice_exp=dice_exp

    button.on_click(on_button_clicked)
    
    #DISPLAYING LAST 3 WIDGETS
    display(HBox([tune_proxdiv1, num_exp,button]))
    display(outtester)
    display(button_output)
     

In [4]:
counterfactual_explorer(dataset,ann_model,feature_names,cont_feat,outcome_name)

NameError: name 'ann_model' is not defined